In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import operator
import seaborn as sns
import json
import matplotlib.pyplot as plt
from gensim import corpora, models

In [2]:
def load_json_to_df(datapass):
    data = [] 
    with open(datapass) as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

In [3]:
business = load_json_to_df("../../../dataset/business.json")

In [4]:
review = load_json_to_df("../../../dataset/review.json")

## Overview:
Train the matrix of business subscores by minimizing a loss function for each restaurant, defined by sum((rec - rating)^2), where rec is (rating subscores dot user preference).
We obtain user preference by running an LDA on all the text reviews, that is, we are setting the preference column as a constant in the loss function. The weight for each topic is calculated by normalizing the sum of the probabilty that each word of one user's texts occurs in the topics generated by LDA on all reviews.

In [132]:
review.head(5)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


## Preprocessing:
0. Extract all the restaurant reviews.
1. tokenize
2. stop words
3. stemmize
4. get only nouns and adjs

In [9]:
is_rest = []
for i in business['categories']:
    
    if 'Restaurants' in i or 'Food' in i:
        is_rest.append(True)
    else:
        is_rest.append(False)
restaurants = business.loc[is_rest]
restaurants.shape

(65028, 15)

In [10]:
rest_id = restaurants['business_id']
rest_review = review.loc[review['business_id'].isin(rest_id)]
rest_review.shape

(3216548, 9)

In [24]:
groupby_user = rest_review.groupby('user_id').size().reset_index(name='counts')

In [134]:
groupby_user.sort_values('counts',ascending=False)

,user_id,counts
186623,CxDOIDnH8gp9KXzpBHJYXw,2828
525915,bLbSNkLggFnqwNNzzq-Ijw,1473
191626,DK57YibC5ShBmqQl97CKog,1021
352049,PKEzKWv_FktMm2mGPjwd0Q,1001
555828,d_TBs6J3twMy9GChqUEXkg,991
539407,cMEtAiW60I5wE_vLfTxoJQ,934
205236,ELcQDlf69kb-ihJfxZyL0A,922
415516,U4INQZOPSUaj8hMjLlZ3KA,791
421891,UYcmGbelzRa0Q6JqzLoguw,790
365144,QJI9OSEn6ujRCtrX06vs1w,741


In [49]:
#preprocess
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from nltk.corpus import stopwords
en_stop = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

def preprocess(text):
    raw = text.lower()
    tokens = tokenizer.tokenize(raw)
    
    pospeech=[]
    tag = nltk.pos_tag(tokens)
    for j in tag:
        if j[1] == 'NN' or j[1] == 'JJ':
            pospeech.append(j[0])
    # remove stop words from tokens
    stopped_tokens = [i for i in pospeech if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    return stemmed_tokens



In [37]:
top1user = rest_review.loc[rest_review['user_id'] == 'CxDOIDnH8gp9KXzpBHJYXw']

In [50]:
texts = []
for t in top1user['text']:
    texts.append(preprocess(t))

## Generating LDA 
For demo purpose, running LDA on only one user.

In [111]:
from gensim import corpora, models


def getlda(df):
    texts = []
    for i in df['text']:
        texts.append(preprocess(i))
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    # generate LDA model
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)
    return ldamodel

## Calculate preference maxtrix
Add up the probabity that each word in the corpus in each topic, then add up for each topic and normalize.

In [104]:
def getprefer(userid, ldamodel):
    user_reviews = rest_review.loc[rest_review['user_id'] == userid]
    l = [0.0,0.0,0.0,0.0,0.0]
    texts = []
    for t in user_reviews['text']:
        texts.append(preprocess(t))
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    for i in ldamodel.get_document_topics(corpus):
        for topic in i:
            l[topic[0]] += topic[1]
    topic_likelihood = []
    for i in l:
        topic_likelihood.append(i/sum(l))
    return topic_likelihood

In [52]:
print(ldamodel.print_topics(num_topics=5, num_words=10))

[(0, '0.015*"sweet" + 0.015*"good" + 0.014*"burger" + 0.013*"chicken" + 0.012*"chocol" + 0.011*"littl" + 0.009*"tast" + 0.008*"bakeri" + 0.007*"hot" + 0.007*"great"'), (1, '0.024*"food" + 0.017*"good" + 0.012*"restaur" + 0.012*"order" + 0.010*"menu" + 0.010*"littl" + 0.009*"time" + 0.007*"meat" + 0.007*"dinner" + 0.007*"fresh"'), (2, '0.026*"coffe" + 0.012*"area" + 0.012*"nice" + 0.012*"place" + 0.011*"bar" + 0.011*"beer" + 0.011*"great" + 0.010*"patio" + 0.010*"good" + 0.009*"littl"'), (3, '0.023*"lunch" + 0.021*"noodl" + 0.021*"soup" + 0.019*"good" + 0.016*"sushi" + 0.015*"rice" + 0.014*"spici" + 0.013*"restaur" + 0.012*"food" + 0.011*"beef"'), (4, '0.035*"tea" + 0.014*"ice" + 0.012*"good" + 0.011*"cream" + 0.011*"thai" + 0.009*"place" + 0.009*"littl" + 0.009*"free" + 0.008*"food" + 0.008*"sweet"')]


In [61]:
print (list(ldamodel.get_document_topics(corpus)))

[[(1, 0.98160475661044855)], [(0, 0.10351247914415389), (2, 0.39406203028790038), (3, 0.4853334948244446)], [(1, 0.86243603582336981), (3, 0.13233299126909179)], [(1, 0.61393731982076083), (4, 0.37072098748824345)], [(0, 0.96602692804771273)], [(1, 0.040552647813302103), (2, 0.94776755714374505)], [(0, 0.099527157129806212), (3, 0.63761176007205556), (4, 0.25983361228581969)], [(1, 0.47142654231967068), (2, 0.51288058757849331)], [(1, 0.41846648151218291), (2, 0.23154430419468139), (4, 0.34109641060796481)], [(1, 0.70303972025812134), (2, 0.28519274256579991)], [(2, 0.96750972984887351)], [(1, 0.68231245748988212), (3, 0.30998321868704426)], [(1, 0.84393624683144108), (3, 0.13426457301872669)], [(1, 0.89782239104488704), (3, 0.078630342108474388)], [(0, 0.096428241202251172), (1, 0.43174177029111654), (2, 0.42077424208689784), (4, 0.045084270130818137)], [(0, 0.015601348085133528), (1, 0.44144307229381596), (2, 0.278132883719311), (3, 0.015803655150525141), (4, 0.24901904075121445)], [

In [65]:
ldamodel.print_topics(12)

[(0,
  '0.015*"sweet" + 0.015*"good" + 0.014*"burger" + 0.013*"chicken" + 0.012*"chocol" + 0.011*"littl" + 0.009*"tast" + 0.008*"bakeri" + 0.007*"hot" + 0.007*"great"'),
 (1,
  '0.024*"food" + 0.017*"good" + 0.012*"restaur" + 0.012*"order" + 0.010*"menu" + 0.010*"littl" + 0.009*"time" + 0.007*"meat" + 0.007*"dinner" + 0.007*"fresh"'),
 (2,
  '0.026*"coffe" + 0.012*"area" + 0.012*"nice" + 0.012*"place" + 0.011*"bar" + 0.011*"beer" + 0.011*"great" + 0.010*"patio" + 0.010*"good" + 0.009*"littl"'),
 (3,
  '0.023*"lunch" + 0.021*"noodl" + 0.021*"soup" + 0.019*"good" + 0.016*"sushi" + 0.015*"rice" + 0.014*"spici" + 0.013*"restaur" + 0.012*"food" + 0.011*"beef"'),
 (4,
  '0.035*"tea" + 0.014*"ice" + 0.012*"good" + 0.011*"cream" + 0.011*"thai" + 0.009*"place" + 0.009*"littl" + 0.009*"free" + 0.008*"food" + 0.008*"sweet"')]

In [91]:
l = [0.0,0.0,0.0,0.0,0.0]
for i in ldamodel.get_document_topics(corpus):
    for topic in i:
        l[topic[0]] += topic[1]

In [94]:
topic_likelihood = []
for i in l:
    topic_likelihood.append(i/sum(l))
topic_likelihood

[0.1192815164564922,
 0.30378618797065166,
 0.18947133864515303,
 0.23097418844631903,
 0.15648676848138396]

In [98]:
print (ldamodel.log_perplexity(corpus))

-7.2268821578


In [136]:
getprefer('CxDOIDnH8gp9KXzpBHJYXw',ldamodel)

[0.17944614938088552,
 0.21964205607806961,
 0.13908729577668794,
 0.11394539512031555,
 0.34787910364404134]

In [113]:
lda = getlda(top1user)

In [114]:
lda.print_topics(12)

[(0,
  '0.017*"beer" + 0.015*"bar" + 0.012*"drink" + 0.012*"good" + 0.011*"patio" + 0.011*"time" + 0.009*"food" + 0.008*"area" + 0.008*"great" + 0.008*"place"'),
 (1,
  '0.018*"noodl" + 0.017*"good" + 0.016*"coffe" + 0.014*"soup" + 0.011*"restaur" + 0.011*"place" + 0.010*"food" + 0.009*"beef" + 0.009*"hot" + 0.009*"nice"'),
 (2,
  '0.021*"tea" + 0.017*"sweet" + 0.013*"good" + 0.012*"littl" + 0.011*"chocol" + 0.010*"ice" + 0.010*"cream" + 0.009*"great" + 0.009*"bakeri" + 0.008*"tast"'),
 (3,
  '0.022*"food" + 0.015*"dim" + 0.014*"meat" + 0.013*"sum" + 0.012*"good" + 0.012*"pork" + 0.010*"bbq" + 0.008*"duck" + 0.008*"order" + 0.007*"littl"'),
 (4,
  '0.020*"lunch" + 0.018*"good" + 0.018*"food" + 0.014*"chicken" + 0.013*"sushi" + 0.011*"littl" + 0.011*"fish" + 0.011*"restaur" + 0.011*"spici" + 0.010*"fresh"')]

## Minimize loss function
<img src="formula.png">
Since we setting the topics for each user as constant, the corpus likelihood here is a constant so we only need to deal with rating error.
We used Sequential Least SQuares Programming (SLSQP) to minize the function.
Intial guess was randomized between 1.0 and 5.0
Might need to explore bias and hyperparameters in the future.

In [ ]:
from scipy.optimize import least_squares
def minimize(prefer, actual_rating)
    dim = 5

    bound = []
    for i in range(0,dim):
        bound.append((1.0,5.0))
    bnds = tuple(bound)


    def f(x,prefer,actual_rating):

        return sum((np.dot(x,prefer) - actual_rating)**2)
    initial_guess = [np.random.uniform(1.0,5.0,dim)]
    result = optimize.minimize(f, initial_guess, args=(prefer,actual_rating), method='SLSQP', bounds=bnds)
    if result.success:
        fitted_params = result.x
        print(fitted_params)
        print(result.fun)
        print(result.nit)
        return result
    else:
        raise ValueError(result.message)

## Combine everything
For each restaurant, run the algorithm to get the subscores, which will take a couple days on MacBook ~~Pro~~.

In [129]:
def add_prefer_to_df():
    d = {}
    for u in groupby_user['user_id']:
        d[u] = getprefer(u,ldamodel)
    rest_review['preference'] = d[df['user_id']]

In [130]:
def train_rest_subscore(bizid):
    biz = rest_review.loc[rest_review['business_id'] == bizid]
    rating = rest_review['stars']
    preference = rest_review['preference']
    result = minimize(preference, rating)
    return result.x